In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [2]:
!pip install jsonlines datasets git+https://github.com/google-research/bleurt.git evaluate sacrebleu

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-gsw3o2m0
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-gsw3o2m0
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [3]:
import jsonlines
data_PATH = drive_PATH + '/data-main/exp2.4.tren-n128-t1-e0.02.jsonl'
data = []

with jsonlines.open(data_PATH) as f:
    for line in f.iter():
        data.append(line)

In [4]:
import tqdm
import jsonlines
import sys
import numpy as np

# Eval - BLEURT
# from datasets import load_metric
# bleurt = load_metric("bleurt", module_type="metric")
# Eval - chrf(++)
import evaluate
chrf = evaluate.load("chrf") # (word_order=2) for chrf++

In [5]:

def CHARF_mbrd(candidates):
  k_samples = len(candidates)
  score_matrix = np.zeros((k_samples, k_samples))

  for j1 in range(k_samples):
    for j2 in range(k_samples):
      references=[candidates[j1][0]]
      predictions=[candidates[j2][0]]
      scores = chrf.compute(predictions=predictions, references=references, word_order=2)['score']
      score_matrix[j2,j1] = scores

  # remove votes for self
  for j1 in range(k_samples):
    for j2 in range(k_samples):
      if j1 == j2:
        score_matrix[j2,j1] = 0

  sum_scores = np.sum(score_matrix, axis=1)
  index = np.argmax(sum_scores)
  mbr_winner = candidates[index]

  return mbr_winner, index, sum_scores, score_matrix

In [6]:
dataset_name = 'TREN'
dataset = data
num_examples = 1000
num_candidates = 128
assert(num_examples == len(dataset))
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]

    chrf_scores = []
    for i in range(num_candidates):
        chrf_scores.append(chrf.compute(predictions=[predictions[i]], references=[references[i]], word_order=2)['score'])
    dataset[j]['CHRF_cand_scores'] = np.array(chrf_scores, dtype=float).tolist()
    # MBR CHRF
    mbr_winner, index, sum_scores, score_matrix = CHARF_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_CHRF_winner'] = mbr_winner
    dataset[j]['MBR_CHRF_winner_index'] = int(index)
    dataset[j]['MBR_CHRF_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_CHRF_score_matrix'] = score_matrix.tolist()

chrf_dataset_PATH = drive_PATH+ f'/data-main/chrf-{dataset_name}.jsonl'
with jsonlines.open(chrf_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)

100%|██████████| 1000/1000 [20:51:48<00:00, 75.11s/it]
